[View in Colaboratory](https://colab.research.google.com/github/NgoDinh/Data-science/blob/master/odoo_bc_xuat_ton_o.ipynb)

In [0]:
# -*- coding: utf-8 -*-
##############################################################################
#
#    OpenERP, Open Source Management Solution
#
##############################################################################
import time
import os
import base64
import pandas as pd
import psycopg2 as pg
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from sqlalchemy import create_engine
from openerp.osv import fields, osv
from openerp.tools.translate import _
from openerp.tools import DEFAULT_SERVER_DATE_FORMAT, DEFAULT_SERVER_DATETIME_FORMAT
from dateutil import parser

class report_online_performance(osv.osv_memory):
    
    _name = "report.online.performance"
    _columns = {
        'date_start': fields.date('Date From', required=True),
        'date_end': fields.date('Date To', required=True),
        'file_output':fields.binary(string="File Output",invisible=True, readonly=True, help='Output file in xlsx format'),
        'file_name':fields.char(string='File Name', invisible=True)
    }
    _defaults = {
         'date_start': time.strftime('%Y-%m-01'),
         'date_end': lambda *a: str(datetime.now() + relativedelta(months=+1, day=1, days=-1))[:10],
    }
    
    def print_excel(self, cr, uid, ids, context=None):
        #set date to query_data
        #============================================================================================#        
        
        def first_day_of_month(any_day):
            any_day = parser.parse(any_day)
            return (any_day.replace(day=1)).strftime("%Y-%m-%d %H:%M:%S")

        def last_day_of_month(any_day):
            any_day = parser.parse(any_day)
            next_month = any_day.replace(day=28, hour=11, minute=59, second=59) + timedelta(days=4)
            return (next_month - timedelta(days=next_month.day)).strftime("%Y-%m-%d %H:%M:%S")
        
        datas = {'ids': context.get('active_ids', [])}
        date_start = self.read(cr, uid, ids, ['date_start'], context=context)
        date_start = first_day_of_month(date_start)
        date_end   = self.read(cr, uid, ids, ['date_end'], context=context)
        date_end   = last_day_of_month(date_end)
        
        out_path = '/tmp/bc_ton_size_%s.xlsx'%(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        engine = pg.connect('postgresql://odoo:odoo@***:***/***')
        
        #Run report
        #============================================================================================# 
        def get_data(cat, date_start, date_end, engine):
            def get_product_id (cat):

                product_name = {'women': ('DEP NU','GIAY NU','SANDAL NU'), 'men': ('DEP NAM','GIAY NAM'), 'other':('PHU KIEN', 'PHU KIEN')}

                query='''
                    SELECT name,id 
                    FROM product_category 
                    WHERE parent_id in
                        (SELECT id 
                        FROM product_category 
                        WHERE name in %s) %s
                    ORDER BY name
                '''

                if cat == 'kid':
                    query = '''
                        SELECT name,id 
                        FROM product_category 
                        WHERE  (name like '%DEP%' AND parent_id in
                                (SELECT id 
                                FROM product_category 
                                WHERE name in ('TRE EM')))
                            OR 
                              name like 'SANDAL (KSD)'
                            OR
                               parent_id in
                                        (SELECT id 
                                        FROM product_category 
                                        WHERE name in ('GIAY TRE EM'))
                        ORDER BY name
                    '''
                elif cat == 'men':
                    query = query%(product_name['men'], "or name LIKE '%%MSD%%'")
                else:
                    query = query%(product_name[cat], "")
                return query

            df = pd.read_sql(get_product_id(cat),engine)

            categ = tuple(df['id'])
            dates = parser.parse(date_end) + relativedelta(days=+1)
            dates = dates.strftime('%Y-%m-%d %H:%M:%S')

            get_inventory ='''
                    SELECT foo.categ_id, sum(foo.qty)::int as inventory
                    FROM 
                    (SELECT
                        quant.id AS id,
                        quant.product_id AS product_id,
                        quant.location_id AS location_id,
                        quant.qty AS qty,
                        quant.lot_id AS lot_id,
                        quant.package_id AS package_id,
                        quant.in_date AS in_date,
                        quant.company_id,
                        template.categ_id AS categ_id,
                        template.name as name_template,
                        prod.default_code AS default_code
                    FROM stock_quant AS quant
                    JOIN product_product prod ON prod.id = quant.product_id
                    JOIN product_template template
                        ON template.id = prod.product_tmpl_id) foo
                    WHERE foo.categ_id in %s
                        AND foo.location_id in (select id from stock_location where usage = 'internal' and name != 'OUTLET') 
                        AND foo.in_date < '%s'
                        AND foo.name_template NOT LIKE '%sQT%s' AND foo.default_code NOT LIKE '%sQT%s'
                    Group by 1
            '''%(categ,dates,'%','%','%','%')

            get_purchase ='''
                SELECT
                template.categ_id,
                sum(stock.product_uom_qty)::int as purchase
                FROM stock_move stock
                    JOIN product_product prod ON prod.id = stock.product_id
                    JOIN product_template template ON template.id = prod.product_tmpl_id
                WHERE stock.location_id in (select id from stock_location where usage ='supplier') 
                    AND stock.location_dest_id in (19)
                    AND (date(timezone('UTC',stock.date::timestamp)) BETWEEN '%s' and '%s')  
                    AND stock.state in ('done')
                    AND template.categ_id in %s
                    AND template.name NOT LIKE '%sQT%s' AND prod.default_code NOT LIKE '%sQT%s'
                Group By 1
            '''%(date_start,date_end,categ,'%','%','%','%')

            get_sale ='''
                SELECT
                pt.categ_id,
                sum(sol.product_uom_qty)::int AS sales
                FROM sale_order_line sol
                    LEFT JOIN sale_order so ON so.id = sol.order_id
                    LEFT JOIN product_product pp ON pp.id = sol.product_id
                    LEFT JOIN product_template pt on (pp.product_tmpl_id=pt.id)
                WHERE so.state NOT IN ('draft', 'sent', 'cancel') and (date(timezone('UTC',so.date_order::timestamp)) BETWEEN '%s' and '%s') and pt.categ_id in %s
                    AND pt.name NOT LIKE '%sQT%s' AND pp.default_code NOT LIKE '%sQT%s'
                Group by 1
            '''%(date_start,date_end,categ,'%','%','%','%')

            get_pos ='''
                SELECT pt.categ_id, sum(pol.qty)::int AS pos
                FROM pos_order_line pol
                    LEFT JOIN pos_order po ON po.id = pol.order_id
                    LEFT JOIN product_product pp ON pp.id = pol.product_id
                    LEFT JOIN product_template pt on (pp.product_tmpl_id=pt.id)
                WHERE po.state IN ('paid', 'done', 'invoiced') and (date(timezone('UTC',po.date_order::timestamp)) BETWEEN '%s' and '%s') and pt.categ_id in %s
                    AND pt.name NOT LIKE '%sQT%s' AND pp.default_code NOT LIKE '%sQT%s'
                Group by 1
            '''%(date_start,date_end,categ,'%','%','%','%')

            df_1 = pd.read_sql(get_inventory,engine)
            df_2 = pd.read_sql(get_purchase,engine)
            df_3 = pd.read_sql(get_sale,engine)
            df_4 = pd.read_sql(get_pos,engine)

            pd.options.display.float_format = '{:,.0f}'.format
            df = df.set_index('id')
            df = df.join(df_1.set_index('categ_id'))
            df = df.join(df_2.set_index('categ_id'))
            df = df.join(df_3.set_index('categ_id'))
            df = df.join(df_4.set_index('categ_id'))
            df = df.fillna(0)
            df = df.sort_values(['inventory'], ascending=False)
            df['total'] = df['sales'] + df['pos'] 

            total = df.apply(np.sum)
            total['name'] = 'Subtotal'
            df = df.append(pd.DataFrame(total.values, index=total.keys()).T, ignore_index=True)

            return df
        
        def get_total(df, opts=0):
            total = df.apply(np.sum)
            total['name'] = 'Total'
            df = df.append(pd.DataFrame(total.values, index=total.keys()).T, ignore_index=True)
            return df

        def run_report(date_start, date_end, out_path, engine):
            men = get_data('men', date_start, date_end, engine)
            women = get_data('women', date_start, date_end, engine)
            kid = get_data('kid', date_start, date_end, engine)
            other = get_data('other', date_start, date_end, engine)
            final = pd.concat([men, women, kid, other])
            final = final.reset_index(drop=True)

            men_sp = men[(men['name'] =='GIÀY CASUAL (MCA)') | (men['name'] =='GIÀY SNEAKER (MSN)') | (men['name'] =='GIAY THE THAO (MTT)')]
            women_sp = women[(women['name'] =='GIÀY CASUAL (WCA)') | (women['name'] =='GIÀY SNEAKER (WSN)') | (women['name'] =='GIAY THE THAO (WTT)')]

            final = get_total(final,1)
            men_sp = get_total(men_sp)
            women_sp = get_total(women_sp)

            writer = pd.ExcelWriter(out_path , engine='xlsxwriter')
            men_sp.to_excel(writer,sheet_name = 'Final_report', startrow=6, startcol=0)
            women_sp.to_excel(writer,sheet_name = 'Final_report', startrow=12,  startcol=0)
            final.to_excel(writer,sheet_name = 'Final_report', startrow=18, startcol=0)

            workbook  = writer.book
            worksheet = writer.sheets['Final_report']

            bold = workbook.add_format({'bold': True, 'border':1})

            big_size = workbook.add_format({'bold': True})
            big_size.set_font_size(15)

            worksheet.set_column('B:B', 35)
            worksheet.set_column('C:G', 20)
            worksheet.write('B3', 'Từ ngày:', bold)
            worksheet.write('B4', 'Đến ngày:', bold)
            worksheet.write('B2', 'BÁO CÁO SỐ LƯỢNG TỒN & BÁN', big_size)
            worksheet.write('C3', date_start, bold)
            worksheet.write('C4', date_end, bold)

            len_men = len(men)
            len_women = len(women) + len_men
            len_kid = len(kid) + len_women
            len_other = len(other) + len_kid

            sub_total_format = workbook.add_format({'bold': True, 'font_color': 'green'})
            sub_total_format.set_bg_color('#C0C0C0')

            total_format = workbook.add_format({'bold': True, 'font_color': 'black'})
            total_format.set_bg_color('#C4D79B')

            header_format = workbook.add_format({'bold': True, 'font_color': 'black'})
            header_format.set_bg_color('#B7DEE8')

            for y in (len_men, len_women, len_kid, len_other):
                for x in range(1,7):
                    worksheet.write(18 + y, x, final.iloc[y-1][x-1], sub_total_format)

            for x in range(2,7):
                worksheet.write(10, x, men_sp.iloc[3][x-1], total_format)

            for x in range(2,7):
                worksheet.write(16, x, women_sp.iloc[3][x-1], total_format)

            for x in range(2,7):
                worksheet.write(19 + len_other, x, final.iloc[len_other][x-1]*(0.5), total_format)

            for x in range(0,6):
                worksheet.write(6, x+1, final.columns.values[x], header_format)

            for x in range(0,6):
                worksheet.write(12, x+1, final.columns.values[x], header_format)

            for x in range(0,6):
                worksheet.write(18, x+1, final.columns.values[x], header_format)

            writer.save()
        
        #run report
        run_report(date_start, date_end, out_path, engine)
        
        #Encode final file to binary and download it.
        #============================================================================================#
        
        PREVIEW_PATH = out_path
        encoded_string = ""     
        with open(PREVIEW_PATH, "r") as image_file:
            encoded_string = base64.encodestring(image_file.read())#.b64encode
        for order in self.browse(cr, uid, ids[0]):
            order.file_name = '%s.xlsx'%(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
            order.file_output = encoded_string
            data_id =  order.id
         
        return {
            'type': 'ir.actions.act_window',
            'res_model': 'report.online.performance',
            'view_mode': 'form',
            'view_type': 'form',
            'res_id': data_id,
            'views': [(False, 'form')],
            'target': 'new',
        }

# vim:expandtab:smartindent:tabstop=4:softtabstop=4:shiftwidth=4:
